<a href="https://colab.research.google.com/github/rnjsdn12332/GAN_study/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [41]:
EPOCHS=300
BATCH_SIZE=128

In [42]:
train_loader = torch.utils.data.DataLoader(
    #png 형식의 컬러 이미지들은 투명도까지 해서 채널 4를 가진다.
    datasets.CIFAR10('./.data',
                          train=True, 
                          download=True,
                          transform=transforms.Compose([
                              transforms.RandomCrop(32,padding=4),#과적합 방지
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
                          ),
    batch_size=BATCH_SIZE, shuffle=True
)

test_loader=torch.utils.data.DataLoader(
     datasets.CIFAR10('./.data',
                          train=False, 
                          download=True, 
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,),(0.3081,))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True

)

Files already downloaded and verified
Files already downloaded and verified


ResNet의 핵심은 네트워크를 작은 블록인 Residual 블록으로 나누었다는 것
Residual 블록의 출력에 입력하였던 x를 더함으로써 모델을 훨씬 길게 설계할 수 있도록 함. 입력과 출력의 관계를 바로 학습하기 보다 입력과 출력의 차이를 따로 학습하는게 성능에 더 좋다는 가설


In [43]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) #배치정규화는 학습률을 너무 높게 잡으면 기울기가 소실되거나 발산하는 증상을 완화시켜줌
        # 각계층에 들어가는 입력을 평균과 분산으로 정규화함으로써 학습을 효율적으로 만듬

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


입력 x가 들어와 컨볼루션 ,배치정규화, 활성화 함수를 거치고, 다시 입력 x를 self.shortcut 을 거치게 하여 크기를 같게 하고 활성화 함수를 거친 값을 더함
그리고 이 값을 relu함수를 통과 시켜 최종 출력을 만듬

In [44]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x))) #RGB 3색을 16개 채널로 만듦
        out = self.layer1(out) # ->  16X32X32
        out = self.layer2(out)# ->  32X16X16
        out = self.layer3(out) # -> 64X8X8
        out = F.avg_pool2d(out, 8) #평균 풀링으로 텐서에 있는 원소의 개수 64개로
        out = out.view(out.size(0), -1) 
        out = self.linear(out)
        return out

In [45]:
model = ResNet()
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) #학습률 감소(learning rate decay)기법
#step_size=50을통해 에폭이 50번 호출 될 때마다 학습률 0.1을 곱함

학습률 감소는 학습이 지냉되면서 최적화 함수의 학습률을 점점 낮춰서 더 정교하게 최적화함.

In [46]:

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data, target
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data, target
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step() #scheduler.step을 해줘서 학습률 감소 적용
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] Test Loss: 2.4098, Accuracy: 24.23%
[2] Test Loss: 2.0686, Accuracy: 43.47%
[3] Test Loss: 2.4465, Accuracy: 40.35%
[4] Test Loss: 2.9886, Accuracy: 36.32%
[5] Test Loss: 1.4594, Accuracy: 56.77%
